In [64]:
import dill

import numpy, scipy, scipy.optimize
import matplotlib
from mpl_toolkits.mplot3d import  Axes3D
from matplotlib import cm
import matplotlib.pyplot as plt
from collections import defaultdict

In [2]:
%matplotlib widget

In [3]:
dill.load_session('sessions/Discrete_BS_Q_Learning_vectorized_2.db')

In [4]:
print(Q)

defaultdict(<function <lambda> at 0x000001D05649E798>, {(0, 87.5): array([4.55559785, 4.55879018, 4.56004045, 4.56092018, 4.56112152,
       4.56105871, 4.56245046, 4.56087956, 4.55984342, 4.55834892,
       4.55608778]), (1.0, 87.5): array([4.5113891 , 4.51307981, 4.51564639, 4.5154327 , 4.51654187,
       4.51672473, 4.51675237, 4.51514481, 4.51434676, 4.51279923,
       4.51217351]), (1.0, 97.5): array([4.61964243, 4.6214235 , 4.62281749, 4.62409212, 4.62400805,
       4.62368725, 4.62377404, 4.62442104, 4.6226707 , 4.62174927,
       4.61857263]), (1.0, 92.5): array([4.56739429, 4.56862733, 4.57037629, 4.57119639, 4.57174105,
       4.57237224, 4.57159734, 4.57080128, 4.56981019, 4.56956439,
       4.56723585]), (2.0, 82.5): array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), (2.0, 107.5): array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), (2.0, 87.5): array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), (2.0, 97.5): array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), (2.0, 77.

In [5]:
def plot_q_values(Q, actions):
    '''Creates a 3d Wireframe plot of the Q-value function for each state-action pair 
       and adds the predicted action (i.e. argmax_a Q(s,a)
    
    Args:
    :params: Q [dict] A dictionary containing the action-values for each state
    :params: actions [np.array] A np.array containing the possible actions
    '''
    
    
    def fun(x, y, Q, actions, t):
        '''Help function used in plot_q_values'''
        # Returns the Q-values for each state-action pair at time step .
        return np.array([Q[(t,wealth)][encode_action(action, actions)] for action, wealth in zip(x,y)])
    
    times = sorted(list(set([t for t,_ in Q.keys()])))
    for time in times:
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        x = actions
        y = np.array(sorted([wealth for t, wealth in Q.keys() if t == time]))
        X, Y = np.meshgrid(x, y)
        zs = np.array(fun(np.ravel(X), np.ravel(Y), Q, actions, time))
        Z = zs.reshape(X.shape)

        # Predicted Actions for each state
        states = [key for key in Q.keys() if key[0] == time]
        predicted_actions = [decode_action(np.argmax(Q[state]), actions) for state in states]
        wealths = [wealth for _, wealth in states]
        predicted_Q_values = [Q[state][np.argmax(Q[state])] for state in states]

        ax.plot_wireframe(X, Y, Z, color="black")

        ax.set_xlabel('investment in risky asset')
        ax.set_ylabel('wealth')
        ax.set_zlabel('Q-values')
        ax.scatter(predicted_actions, wealths, predicted_Q_values, zdir="z", c="red", alpha=1, label="Predicted Actions")
        plt.title("Learned Q-value surface (at t={})".format(time))
        ax.legend()

        plt.show()

In [6]:
plot_q_values(Q, actions)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
def dict_to_list(Q, actions):
    '''Converts the action-value table Q from a dictionary to a list of four numpy arrays.
    
    Args:
    - Q:[dict] Q-Table
    - actions:[np.array] possible risky asset allocations
    
    Returns a list of four np.arrays:
        - time
        - action
        - wealth
        - Q-value
    '''
    
    tData = np.array([t for t,_ in Q.keys()]).repeat(len(actions))
    aData = np.array(list(actions)*(len(tData)//len(actions)))
    vData = np.array([V for _,V in Q.keys()]).repeat(len(actions))
    qData = np.array([values for values in Q.values()]).reshape(len(tData))
    
    return [tData, aData, vData, qData]

In [23]:
tData, aData, vData, qData = dict_to_list(Q, actions)
print(tData)
print(vData)
print(aData)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.

In [65]:
def lists_to_dict(tData, vData, qData):
    '''Creates the action-value as dictionary from three lists.
    
    Args:
    - tData[list[float]]: time values
    - vData[list[float]]: wealth data
    - qData[list[float]]: action values
    
    Returns:
    - Q[dict]: keys are tuples (t,v) and values are lists of length nr. of actions with respective action-values.
    '''
    
    # Create dict keys as tuples (t,v) -> list of tuples
    keys = list(dict.fromkeys(zip(tData, vData)))
    
    # Create dict values as action-value per action -> list of np.arrays of length nr. actions
    values = np.split(np.array(qData), indices_or_sections = len(keys))
    
    # Create dict
    Q = defaultdict(lambda: np.zeros(env.action_space.n))
    for key, value in list(zip(keys, values)):
        Q[key] = value
    
    return Q

In [11]:
def rm_high_low_wealths_from_list(Q_list):
    '''Reduces the Q-value list by their highest and lowest values for function fitting. 
       (Hightest and lowest buckets are not accurate.)
       
    Args:
    - Q_list:[list] list of four lists t, a, v, q from dict_to_list
    
    Returns a list of four np.arrays:
        - time
        - action
        - wealth
        - Q-value
    '''
    
    # Unpack values
    tData, aData, vData, qData = Q_list
    
    # Remove lowest/highest wealth values
    tData = tData[(vData!=min(vData)) & (vData!=max(vData))]
    aData = aData[(vData!=min(vData)) & (vData!=max(vData))]
    qData = qData[(vData!=min(vData)) & (vData!=max(vData))]
    vData = vData[(vData!=min(vData)) & (vData!=max(vData))]
    
    return [tData, aData, vData, qData]

In [78]:
def func(data, a, b, c):
    '''Parametrised action-value function Q in the log-utility case.'''
    
    t   = data[0]
    inv = data[1]
    v   = data[2]
    
    # See master thesis
    z = a * (inv**2) + b * inv + np.log(v) + c * (2-t-1)
    # Q-values for terminal states are zero
    z[t==T]=0
    
    return  z

In [93]:
def structure_preserving_update(Q, actions, step_size, func, initial_params):
    '''Updates the action-value function Q by fitting parameters of function func to Q-table and updating all Q-values
       by a step towards the fitted values.
       
    Args:
    - Q:[dict] Action-value function
    - actions:[np.array] possible risky asset allocations
    - step_size:[float] step_size parameter for an update step towards the fitted values
    - func:[callable] parametrised family of functions used for fitting
    - initial_params:[list[float]] List of initial parameters for the optimisation step (must match the nr. of parameters of func)
    
    Returns:
    - Q:[dict] Update Action-value function
    '''
    
    # Change dict representation to list representation (needed for scipy.optimize)
    Q_list = dict_to_list(Q, actions)
    
    # Remove highest / lowest wealth levels from action-value function (wealth levels are set to lower / upper limit, not midpoints)
    tData, aData, vData, qData = rm_high_low_wealths_from_list(Q_list)
    
    # Fit function
    fittedParameters, pcov = scipy.optimize.curve_fit(func, [tData, aData, vData], qData, p0 = initial_params)
    
    # Fit action-values at grid points
    fittedQValues = func([tData, aData, vData], *fittedParameters)
    
    # difference between predicted values and actual values
    Q_diff = fittedQValues - qData
    
    # Update Q-values
    qData += step_size*Q_diff
    
    # Transform lists to dictionary to dictionary
    Q_new_dict = lists_to_dict(tData, vData, qData)
    
    # Add states with highest lowest wealths (no change)
    old_keys = set(Q.keys())
    new_keys = set(Q_new_dict.keys())
    difference = list(old_keys - new_keys)
    
    for key in difference:
        Q_new_dict[key] = Q[key]
    
    
    return Q_new_dict, fittedParameters    

In [94]:
step_size = 0.1
initial_params = [-1,0,0]


In [95]:
Q_new, fittedParameters = structure_preserving_update(Q, actions, step_size, func, initial_params)

In [96]:
print(Q_new)

defaultdict(<function lists_to_dict.<locals>.<lambda> at 0x000001D05A0B7C18>, {(0.0, 87.5): array([4.55270527, 4.55700472, 4.55930317, 4.561015  , 4.56186314,
       4.56222041, 4.56363365, 4.56212736, 4.56084923, 4.55890543,
       4.55601852]), (1.0, 87.5): array([4.50741407, 4.51036205, 4.51384518, 4.51457293, 4.51623812,
       4.5168165 , 4.51700203, 4.51546276, 4.5143989 , 4.51240738,
       4.51099235]), (1.0, 97.5): array([4.61566342, 4.61869273, 4.62112053, 4.62318777, 4.62377904,
       4.62390412, 4.62414289, 4.62463272, 4.62271181, 4.62128377,
       4.61757292]), (1.0, 92.5): array([4.56337572, 4.5659118 , 4.56865908, 4.57031724, 4.57147436,
       4.57245624, 4.57191949, 4.57111056, 4.56987297, 4.56905301,
       4.56610544]), (2.0, 82.5): array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), (2.0, 107.5): array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), (2.0, 87.5): array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), (2.0, 97.5): array([0., 0., 0., 0., 0., 0., 0., 0.

In [97]:
plot_q_values(Q_new, actions)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …